In [1]:
import numpy as np
import copy

In [2]:
# Example
# Input RNA sequence
s = 'CGGAUACUUCUUAGACGA'
n = len(s)

# Weights of complimentary pairs
w = {'CG': 3,
     'GC': 3,
     'AU': 2,
     'UA': 2,
     'GU': 1,
     'UG': 1}

# Distance parameter
D = 3

In [3]:
# Matrix for storing weights of subsequences
M = np.full((n,n), 0, dtype=int)

In [4]:
# Set the weights of non valid pairs to 0
# Non valid pairs are those that are closer then D
# or not complimentary
def calcG(i, j, s, w):    
    deltaG = 0
    if abs(i - j) < D + 1:
        return 0
    elif s[i]+s[j] not in w:
        return 0
    else: deltaG = w[s[i]+s[j]]
    return deltaG

# Recursion to fill in the matrix of subsequences weights
def calcM(i, j, s, w):    
    if i > j:
        return 0
    else:
        # if nt i is left unpaired
        maxM = calcM(i+1, j, s, w)
        # if nt i is paired
        for k in range(i+D+1, j+1):            
            currM = calcG(i, k, s, w) + calcM(i+1, k-1, s, w) + calcM(k+1, j, s, w)
#             print(f'{currM} = {calcG(i, k, s, w)} + {calcM(i+1, k-1, s, w)} + {calcM(k+1, j, s, w)}')           
            if currM > maxM:
                maxM = currM
                
    return maxM

In [5]:
for i in range(n, -1, -1):
    for j in range(i+D+1, n):
        M[i,j] = calcM(i, j, s, w) 

In [6]:
def printM(M):
    print('\n'.join(' '.join(str(x) for x in n) for n in M))

In [7]:
print(M)

[[ 0  0  0  0  0  0  3  4  4  6  6  6  6  9  9 11 14 14]
 [ 0  0  0  0  0  0  3  4  4  6  6  6  6  7  9 11 11 11]
 [ 0  0  0  0  0  0  3  3  3  5  5  5  5  6  8 10 10 10]
 [ 0  0  0  0  0  0  0  2  2  2  2  4  4  5  7  7  8 10]
 [ 0  0  0  0  0  0  0  0  0  0  2  2  4  5  7  7  8 10]
 [ 0  0  0  0  0  0  0  0  0  0  2  2  2  5  5  5  8  8]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  2  5  5  5  8  8]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  2  3  5  5  6  7]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  2  3  5  5  5  7]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  3  3  3  5  5]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  2  2  3]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  2]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0

In [8]:
# Backtrack for constructing the structure in dot-bracket notation
def backtrack(i, j, M, s, w):
    struct = ""
    struct1 = ""
    struct2 = ""
    if j - i <= D:
        for k in range(j-i+1):
            struct += "."
        print(f'({i},{j}): case 0')
        print(struct)
        return struct
    elif M[i,j] == M[i+1,j]:
        struct = "." + backtrack(i+1,j,M,s,w)
        print(f'({i},{j}): case A')
        print(struct)
        return struct
    elif M[i,j] == M[i+1,j-1] + calcG(i, j, s, w):
        struct = backtrack(i+1,j-1,M,s,w)
        struct = "(" + struct + ")"
        print(f'({i},{j}): case B')
        print(struct)
        return struct 
    else:
        print(f'({i},{j}): case C')        
        for k in range(i+D+1,j):
            if M[i,j] == M[i+1,k-1]+M[k+1,j]+calcG(i, k, s, w):
                struct1 = backtrack(i+1,k-1,M,s,w)
                struct2 = backtrack(k+1,j,M,s,w)
                struct = "(" + struct1 + ")" + struct2
        print(struct)
        return struct

In [9]:
backtrack(0, 17, M, s, w)

(0,17): case C
(2,14): case C
(3,5): case 0
...
(10,12): case 0
...
(9,13): case B
(...)
(8,14): case B
((...))
(7,14): case A
.((...))
(...).((...))
(1,15): case B
((...).((...)))
(17,17): case 0
.
(((...).((...)))).


'(((...).((...)))).'

In [10]:
# stack of suboptimal structures
R = {}
R[1] = {}
print(R)

# set of regions being considered
sigma = []
sigma.append([0,n-1])
# sigma = np.vstack((sigma, [2,n-2]))
# print(sigma[-1])
# region = sigma.pop()
# print(sigma)
# print(region)
# # last = len(sigma)
# sigma = np.delete(sigma, (last-1), axis=0)
# sigma.insert(0,[1,n-2])
print(sigma)

# partial structure
# Sp = np.empty(1, dtype=object)
Sp = {}
print(len(Sp))

#tuple for storing current subopt structure candidate
S = {}
S[1] = (sigma, Sp)
print(S[1])

# set of suboptimal solutions
Sol = []

# residual tolerance
Delta = 2

# index of suboptimal structure
structN = ''

print(structN)


{1: {}}
[[0, 17]]
0
([[0, 17]], {})



In [11]:
# Backtracking for delta-suboptimal structures

def subopts(sigma, Sp, Delta, M, s, structN):
    
    Sp_temp = []
    structN_temp = structN
    
    if len(structN) != 0:
        print(f'INPUT:::{structN}:::{Sp[structN]}')
    
    print(f'initial structN: {structN_temp}')
    
    if len(sigma) == 0:
        
        print(f'sigma: {sigma}')
        print(f'Sp: {Sp}')
        
        if len(structN) != 0:
            print('***************')
            print(f'SRUCTURE ::: {Sp[structN]}')
            Sol.append(Sp[structN])
            print('***************')            
        
        return Sol
    else:
        
        region = sigma.pop()
        i = region[0]
        j = region[1]
        
        if j-i <= D:
            
#             print(f'{j}-{i}<={D}')
#             print(f'sigma: {sigma}')
#             print(f'Sp: {Sp}')            
            
            return subopts(sigma, Sp, Delta, M, s, structN)
        
        else:
                
            delta = M[i,j] - M[i+1,j]

#             print(f'CASE A: {delta}')

            if Delta - delta >= 0:
                
#                 print(f'structN A: {structN}')                
                               
                if len(structN) != 0:
                    Sp_temp = copy.deepcopy(Sp[structN])                  
                
                structN = structN + f'N{i}{j}'
                
#                 print(f'structN A: {structN}')
                
                if Sp_temp:
                    Sp[structN] = copy.deepcopy(Sp_temp)
                else:                    
                    Sp[structN] = [[]]
                    
                if len(structN) != 0:
                    print(f'A :: {structN}:{Sp[structN]}')
                    
                print(f'Sp A: {Sp}')
                    
                sigma.insert(0, [i+1,j])

                subopts(sigma, Sp, Delta - delta, M, s, structN)
                
                structN = structN_temp
            
                print(f'structN A: {structN}')
                
                print(f'sigma: {sigma}')
                print(f'Sp: {Sp}')
                
            for k in range(i+D+1,j+1):
                
                if len(structN) != 0:
                    print(f' CCC {structN}: {Sp[structN]} :: {Sp}')
                
                if k+1<18: 
                    delta = M[i,j] - (M[i+1,k-1] + M[k+1,j] + calcG(i, k, s, w))
                else: 
                    delta = M[i,j] - (M[i+1,k-1] + 0 + calcG(i, k, s, w))
                print(f'CASE C[{k}]: {delta}')

                sigma_temp = sigma
            
                

                if Delta - delta >= 0:
                    sigma_temp.insert(0, [k+1,j])
                    sigma_temp.insert(0, [i+1,k-1])                     
                    
                    if len(structN) != 0:
                        Sp_temp = copy.deepcopy(Sp[structN])
                        
                        print(f'CCCC :: {structN}:{Sp[structN]}')
                    
                    structN = structN + f'Y{i}{k}'
#                     print(f'structN C: {structN}')
                    
                    if Sp_temp:
#                         for keys, value in Sp.items():
#                             print(f'{keys} : {value}')
                            
                        Sp_temp.append([i,k])
                        Sp[structN] = copy.deepcopy(Sp_temp)                        
                        
                        print(f'Sp_temp: {Sp_temp} :: {Sp}')
                        print(f'[{i},{k}]')
                        print(f'C :: {structN}:{Sp[structN]} :: {Sp}')
                    else:
                        Sp[structN] = [[i,k]]                       
                    
                    print(f'Sp C: {Sp}')
                    
                    print(f'k: {k}')
                    print(f'sigma_temp: {sigma_temp}')
                    print(f'Sp: {Sp}')
                    print(f'Delta - delta: {Delta - delta}')
                    
                    
                    subopts(sigma_temp, Sp, Delta - delta, M, s, structN)
                    
                    structN = structN_temp
                    print(f'structN C: {structN}')
            
#         if len(structN_temp) != 0:

#             print(f'Sp before deleting previous Sp: {Sp}')
#             Sp.pop(structN_temp)
#             print(f'Sp after deleting previous Sp: {Sp}')

In [12]:
subopts(sigma, Sp, Delta, M, s, structN)
print(f'sol: {Sol}')

initial structN: 
CASE C[4]: 6
CASE C[5]: 6
CASE C[6]: 7
CASE C[7]: 4
CASE C[8]: 5
CASE C[9]: 7
CASE C[10]: 6
CASE C[11]: 8
CASE C[12]: 8
CASE C[13]: 5
CASE C[14]: 7
CASE C[15]: 5
CASE C[16]: 0
Sp C: {'Y016': [[0, 16]]}
k: 16
sigma_temp: [[1, 15], [17, 17]]
Sp: {'Y016': [[0, 16]]}
Delta - delta: 2
INPUT:::Y016:::[[0, 16]]
initial structN: Y016
INPUT:::Y016:::[[0, 16]]
initial structN: Y016
A :: Y016N115:[[0, 16]]
Sp A: {'Y016': [[0, 16]], 'Y016N115': [[0, 16]]}
INPUT:::Y016N115:::[[0, 16]]
initial structN: Y016N115
 CCC Y016N115: [[0, 16]] :: {'Y016': [[0, 16]], 'Y016N115': [[0, 16]]}
CASE C[6]: 2
 CCC Y016N115: [[0, 16]] :: {'Y016': [[0, 16]], 'Y016N115': [[0, 16]]}
CASE C[7]: 4
 CCC Y016N115: [[0, 16]] :: {'Y016': [[0, 16]], 'Y016N115': [[0, 16]]}
CASE C[8]: 4
 CCC Y016N115: [[0, 16]] :: {'Y016': [[0, 16]], 'Y016N115': [[0, 16]]}
CASE C[9]: 3
 CCC Y016N115: [[0, 16]] :: {'Y016': [[0, 16]], 'Y016N115': [[0, 16]]}
CASE C[10]: 7
 CCC Y016N115: [[0, 16]] :: {'Y016': [[0, 16]], 'Y016N115'